In [ ]:
# 정형 데이터의 끝판왕인 앙상블 학습 알고리즘
# 랜덤 포레스트 -> 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련세트에 과대적합되는 것을 막아주고 검증세트와 테스트 세트에서 안정적인 성능 발휘
# 부스트스트랩 방식 -> 데이터세트에서 중복을 허용하여 데이터를 샘플링하는 방식

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# 교차검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 훈련세트에 과대적합

0.9973541965122431 0.8905151032797809


In [ ]:
# 특성 중요도 확인
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 결정트리에서 본 특성 중요도와 비교 -> 더 많은 특성이 훈련에 기여할 기회를 얻음 -> 과대적합을 줄이고 일반화 성능을 높이는 데 도움

[0.23167441 0.50039841 0.26792718]


In [ ]:
# 부스트스트랩 샘플에 포함되지 않고 남는 샘플 -> OOB샘플
# OOB샘플을 이용하여 부스트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. -> 검증세트의 역할

#oob_score=True로 지정하고 모델을 훈련하여 OOB점수 출력
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)
# OOB점수는 교차검증을 대신할 수 있으며, 이는 훈련세트에 더 많은 샘플을 사용할 수 있도록 함

0.8934000384837406


In [ ]:
# 앙상블 학습 - 엑스트라 트리
# 랜덤포레스트와의 차이점 -> 부스트 스트랩 사용X 즉, 각 결정트리를 만들 때 전체 훈련세트 사용, 가장 좋은 분할을 찾는 것이 아닌 무작위분할
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 랜덤하게 노드를 분할하기 때문에, 빠른 계산 속도가 엑스트라 트리의 장점 but 보통 더 많은 트리 필요

0.9974503966084433 0.8887848893166506


In [ ]:
# 특성 중요도 확인
et.fit(train_input, train_target)
print(et.feature_importances_)
# 엑스트라 트리도 결정트리보다 당도에 대한 의존성이 작다.

[0.20183568 0.52242907 0.27573525]


In [ ]:
# 앙상블 학습 - 그레이디언트 부스팅
# 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 방법
# 과대적합에 강하고 높은 일반화 성능 기대 가능 (얕은 트리 -분할횟수가 적음-데이터의 미세한 잡음까지 반영X-과대적합위험 감소)

# 그레디언트부스팅을 사용해 교차 검증 점수 확인
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [12]:
# 학습률과 트리의 개수를 증가시켜 성능 향상 유도
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
# 특성 중요도 확인
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


In [14]:
# 그래디언트 부스팅은 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있다. but, 순서대로 트리를 추가하기 때문에 훈련속도가 느림,
# 즉 그래디언트 부스팅은 n_jobs 매개변수가 없다.

In [15]:
# 히스토그램 기반 그레디언트 부스팅
# 특성을 구간으로 나눈다(히스토그램) -> 노드를 분할할 떄 최적의 분할을 매우 빠르게 찾을 수 있음
# 256개의 bin중 1개를 결측치 전용으로 따로 떼어놓음 -> 입력시 누락값이 있다면 그 결측치 bin에 속함 -> 따로 전처리 필요X
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
# 히스토그램 기반 그레디언트 부스팅은 자체적으로 특성 중요도 제공X
# permutaion_importance()함수 사용(특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰 -> 특성 중요도 계산)
# n_repeats 변수 : 랜덤하게 섞을 횟수 지정

# 특성 중요도 확인
# 1. 훈련세트
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean) # (중요도, 평균, 표준편차)

[0.08876275 0.23438522 0.08027708]


In [18]:
# 2. 테스트 세트
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [19]:
# 히스토그램 기반 그레디언트 부스팅을 사용해 테스트 세트에서 성능을 최종확인
hgb.score(test_input,test_target)

0.8723076923076923

In [20]:
# 사이킷런에서 제공하는 히스토그램 기반 그레디언트 부스팅 알고리즘을 구현한 XGBoost라이브러리
# tree_method 매개변수를 'hist'로 지정하여 사용

# 교차검증 점수 확인
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


In [21]:
# 히스토그램 기반 그레디언트 부스팅 알고리즘을 구현한 마이크로스프 제작 LightGBM 라이브러리

# 교차검증 점수 확인
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001330 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3152, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 371
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.758057 -> initscore=1.142055
[LightGBM] [Info] Start training from score 1.142055
[LightGBM] [Info] Number o